<h1><center>Ind Vs Pakistan Cricket Hackathon

In [1]:
!pip3 install pyqt5 --upgrade --user
!pip3 install scipy --upgrade --user --no-warn-script-location
!pip3 install numpy --upgrade --user --no-warn-script-location

  Using cached scipy-1.11.3-cp39-cp39-win_amd64.whl (44.3 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.1.0 requires scipy~=1.10.1, but you have scipy 1.11.3 which is incompatible.
pandas-profiling 3.6.3 requires scipy<1.10,>=1.4.1, but you have scipy 1.11.3 which is incompatible.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
sktime 0.21.1 requires numpy<1.26,>=1.21.0, but you have numpy 1.26.0 which is incompatible.
pycaret 3.1.0 requires numpy<1.24,>=1.21, but you have numpy 1.26.0 which is incompatible.
pycaret 3.1.0 requires scipy~=1.10.1, but you have scipy 1.11.3 which is incompatible.
pandas-profiling 3.6.3 requires numpy<1.24,>=1.16.0, but you have numpy 1.26.0 which is incompatible.
pandas-profiling 3.6.3 requires scipy<1.10,>=1.4.1, but you have scipy 1.11.3 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.0 which is incompatible.



  Using cached numpy-1.26.0-cp39-cp39-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [2]:
!pip3 install pycaret --user

  Using cached scipy-1.10.1-cp39-cp39-win_amd64.whl (42.5 MB)
  Using cached numpy-1.23.5-cp39-cp39-win_amd64.whl (14.7 MB)
  Using cached numpy-1.21.6-cp39-cp39-win_amd64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.3
    Uninstalling scipy-1.11.3:
      Successfully uninstalled scipy-1.11.3


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
pandas-profiling 3.6.3 requires scipy<1.10,>=1.4.1, but you have scipy 1.10.1 which is incompatible.


In [ ]:
import sys
sys.path.append('C:/Users/LSRNR/AppData/Roaming/Python/Python39/Scripts')
sys.path.append('C:/Users/LSRNR/AppData/Roaming/Python/Python39/site-packages')

In [17]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pycaret
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.linear_model import BayesianRidge, Ridge
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from pycaret.regression import *

In [5]:
# Reading the dataset into pandas DataFrame
df = pd.read_csv('train-file.csv')
df.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id
0,1,Rohit Sharma,DNB,-,-,1,0,23 Jun 2007,v Ireland Belfast,1
1,1,Rohit Sharma,8,0,3,1,0,26 Jun 2007,v South Africa Belfast,2
2,1,Rohit Sharma,1,-,-,0,0,5 Oct 2007,v Australia Hyderabad,3
3,1,Rohit Sharma,52,-,-,1,0,18 Nov 2007,v Pakistan Jaipur,4
4,1,Rohit Sharma,29,-,-,0,0,3 Feb 2008,v Australia Brisbane,5


In [6]:
df.shape

(2021, 10)

In [7]:
#Understanding the nature of each feature
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_id      2021 non-null   int64 
 1   player_name    2021 non-null   object
 2   runs_scored    2021 non-null   object
 3   wickets        2021 non-null   object
 4   runs_conceded  2021 non-null   object
 5   catches        2021 non-null   object
 6   stumpings      2021 non-null   object
 7   match_date     2021 non-null   object
 8   opposition     2021 non-null   object
 9   match_id       2021 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 158.0+ KB


<b>Here the runs_scored, wickets, runs_conceded, catches and stumpings are to be numerical features. The opposition has two data, one is the team against which India played and the location of the match. Feature transformation has to be done on this feature.

In [8]:
#Knowing each column contents to check for data inconsistencies
df.player_name.value_counts()

Virat Kohli            281
Rohit Sharma           251
Ravindra Jadeja        186
Ravichandran ashwin    115
Babar Azam             108
Mohammed Shami          94
Kuldeep Yadav           90
Hardik Pandya           82
Faqar Zaman             78
Jasprit Bumrah          78
Imam-Ul-Haq             66
Mohammad Rizwan         65
Shadab Khan             64
KL Rahul                61
Shreyas Iyer            47
Shardul Thakur          44
Shaheen Shah Afridi     44
Shubman Gill            35
Mohammad Nawaz          32
Suryakumar Yadav        30
Mohammed Siraj          30
Haris Rauf              28
Ishan Kishan            25
Iftikhar Ahmed          19
Salman Agha             18
Hasan Ali               16
Mohammad Wasim Jr.      16
Usama Mir                8
Saud Shakeel             6
Abdullah Shafique        4
Name: player_name, dtype: int64

In [9]:
print('Total Number of players in this dataset are', len(df.player_name.unique()))

Total Number of players in this dataset are 30


<b>This dataset has both Indian and Pakistani players.

In [10]:
df.runs_scored.unique()

array(['DNB', '8', '1', '52', '29', '0', '39*', '70*', '24', '3*', '66',
       '2', '9', '26', '11', '0*', '22', '58', '22*', '3', '19', '12',
       '37', '32', '25', '18', '54', '31', '11*', '28', '8*', '25*', '4*',
       '15', '60*', '43*', '4', '7', 'TDNB', '48', '57', '36', '114',
       '61*', '101*', '82', '13', '51', '68', '38', '6', '69', '10', '41',
       '20', '5', '44', '23', '87*', '68*', '7*', '81', '86*', '39', '94',
       '55', '72', '90*', '117', '9*', '95', '27', '23*', '21', '80',
       '77', '24*', '30*', '33', '1*', '14', '108', '183', '106', '128*',
       '2*', '31*', '83', '21*', '65', '47*', '58*', '43', '33*', '60',
       '5*', '49*', '46', '102', '48*', '115', '6*', '64*', '42', '61',
       '141*', '100*', '79', '115*', '209', '86', '99', '123', '78',
       '66*', '62*', '14*', '136', '56', '52*', '18*', '10*', '264',
       '139*', '138', '107', '57*', '16*', '64', '44*', '16', '137', '34',
       '63', '150', '171*', '91', '124', '59', '85*', '154*'

In [11]:
df.wickets.unique()

array(['-', '0', '2', '1', '3', '5', '4', '6'], dtype=object)

In [12]:
df.runs_conceded.unique()

array(['-', '3', '5', '4', '11', '9', '13', '7', '22', '40', '34', '27',
       '12', '28', '53', '51', '49', '54', '50', '43', '29', '16', '63',
       '19', '66', '24', '30', '6', '14', '39', '8', '42', '74', '37',
       '33', '59', '62', '17', '41', '48', '32', '15', '47', '10', '58',
       '31', '56', '46', '18', '23', '21', '26', '36', '35', '44', '55',
       '68', '67', '25', '20', '45', '52', '72', '57', '64', '73', '69',
       '61', '84', '81', '38', '71', '75', '1', '60', '70', '2', '76',
       '65', '77', '79', '78', '80', '87', '0', '83', '82'], dtype=object)

In [13]:
df.catches.unique()

array(['1', '0', '-', '3', '2', '4'], dtype=object)

In [14]:
df.stumpings.unique()

array(['0', '-', '1'], dtype=object)

In [15]:
df.columns

Index(['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
       'catches', 'stumpings', 'match_date', 'opposition', 'match_id'],
      dtype='object')

<b>Here, it would be optimal to convert DNB to 0, and since the problem concentrates on how much would a person score, we can remove the * from scores.

In [18]:
for i in ['runs_scored', 'runs_conceded', 'wickets', 'catches','stumpings']:
    for j in range(len(df[i])):
        df[i][j] = str(df[i][j]).replace('TDNB','0')
        df[i][j] = str(df[i][j]).replace('DNB','0')
        df[i][j] = str(df[i][j]).replace('absent','0')
        df[i][j] = str(df[i][j]).replace('*','')
        df[i][j] = str(df[i][j]).replace('-','0')

In [19]:
#Changing the data type of the features
df = df.astype({'runs_scored':'int64','runs_conceded':'int64','wickets':'int64','catches':'int64','stumpings':'int64'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   player_id      2021 non-null   int64 
 1   player_name    2021 non-null   object
 2   runs_scored    2021 non-null   int64 
 3   wickets        2021 non-null   int64 
 4   runs_conceded  2021 non-null   int64 
 5   catches        2021 non-null   int64 
 6   stumpings      2021 non-null   int64 
 7   match_date     2021 non-null   object
 8   opposition     2021 non-null   object
 9   match_id       2021 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 158.0+ KB


In [20]:
df.head(2)

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id
0,1,Rohit Sharma,0,0,0,1,0,23 Jun 2007,v Ireland Belfast,1
1,1,Rohit Sharma,8,0,3,1,0,26 Jun 2007,v South Africa Belfast,2


<b>match_id and match_date are providing the same meaning and therefore match_date can be dropped from the df

In [21]:
df.drop(columns = 'match_date',inplace=True)

In [22]:
df.opposition.unique()

array(['v Ireland Belfast', 'v South Africa Belfast',
       'v Australia Hyderabad', 'v Pakistan Jaipur',
       'v Australia Brisbane', 'v Sri Lanka Brisbane',
       'v Australia Melbourne', 'v Sri Lanka Canberra',
       'v Australia Adelaide', 'v Sri Lanka Adelaide',
       'v Australia Sydney', 'v Sri Lanka Hobart', 'v Pakistan Mirpur',
       'v Bangladesh Mirpur', 'v Hong Kong Karachi', 'v Pakistan Karachi',
       'v Bangladesh Karachi', 'v Sri Lanka Karachi',
       'v Sri Lanka Dambulla', 'v Sri Lanka Colombo (RPS)',
       'v England Rajkot', 'v England Indore', 'v England Kanpur',
       'v England Cuttack', 'v New Zealand Hamilton',
       'v New Zealand Auckland', 'v West Indies Kingston',
       'v West Indies Gros Islet', 'v South Africa Ahmedabad',
       'v Zimbabwe Bulawayo', 'v Sri Lanka Bulawayo', 'v Zimbabwe Harare',
       'v Sri Lanka Harare', 'v Bangladesh Dambulla',
       'v Pakistan Dambulla', 'v New Zealand Dambulla',
       'v New Zealand Bengaluru', 'v N

In [23]:
international_cricket_teams = ["Australia", "Bangladesh", "England", "India", "New Zealand", "Pakistan",
                               "South Africa", "Sri Lanka", "West Indies", "Zimbabwe","Ireland",
                              "Hong Kong", "Afghanistan", "Nepal","Netherlands","U.A.E."]
df['opposition_team'] = ''
for i in range(len(df.opposition)):
    for j in international_cricket_teams:
        if j in df.opposition[i]:
            df['opposition_team'][i] = j

In [24]:
df.opposition_team.unique()

array(['Ireland', 'South Africa', 'Australia', 'Pakistan', 'Sri Lanka',
       'Bangladesh', 'Hong Kong', 'England', 'New Zealand', 'West Indies',
       'Zimbabwe', 'Afghanistan', 'U.A.E.', 'India', 'Nepal',
       'Netherlands'], dtype=object)

In [25]:
pattern = 'Geeks f'
string = 'Geeks for geeks'
last_index = (re.search(pattern, string)).span()[1]

In [26]:
string[last_index:]

'or geeks'

In [27]:
df['Location'] = ''
for i in range(df.shape[0]):
    index_end_loc = (re.search(df['opposition_team'][i],df['opposition'][i])).span()[1]
    df['Location'][i] = df['opposition'][i][index_end_loc + 1:]

In [28]:
df.head(3)

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,opposition,match_id,opposition_team,Location
0,1,Rohit Sharma,0,0,0,1,0,v Ireland Belfast,1,Ireland,Belfast
1,1,Rohit Sharma,8,0,3,1,0,v South Africa Belfast,2,South Africa,Belfast
2,1,Rohit Sharma,1,0,0,0,0,v Australia Hyderabad,3,Australia,Hyderabad


In [29]:
df.Location.unique()

array(['Belfast', 'Hyderabad', 'Jaipur', 'Brisbane', 'Melbourne',
       'Canberra', 'Adelaide', 'Sydney', 'Hobart', 'Mirpur', 'Karachi',
       'Dambulla', 'Colombo (RPS)', 'Rajkot', 'Indore', 'Kanpur',
       'Cuttack', 'Hamilton', 'Auckland', 'Kingston', 'Gros Islet',
       'Ahmedabad', 'Bulawayo', 'Harare', 'Bengaluru', 'Chennai',
       'Durban', 'Johannesburg', 'Cape Town', 'Gqeberha', 'Centurion',
       'Port of Spain', 'North Sound', 'Chester-le-Street',
       'Visakhapatnam', 'Perth', 'Hambantota', 'Pallekele', 'Mohali',
       'Dharamsala', 'Cardiff', 'The Oval', 'Birmingham', 'Pune',
       'Ranchi', 'Nagpur', 'Kochi', 'Napier', 'Wellington', 'Fatullah',
       'Eden Gardens', 'Wankhede', 'Delhi', 'Nottingham', "Lord's",
       'Leeds', 'Dubai (DSC)', 'Guwahati', 'Brabourne',
       'Thiruvananthapuram', 'Mount Maunganui', 'Southampton',
       'Manchester', 'Providence', 'Raipur', 'Bridgetown', 'Tarouba',
       'Lucknow', 'Christchurch', 'Vadodara', 'Gwalior', 'Paarl',


In [30]:
indian_cities_with_international_cricket_grounds = ["Mumbai", "Kolkata", "Chennai", "Bengaluru", "Delhi", 
                                                    "Hyderabad", "Mohali", "Ahmedabad", "Rajkot", "Kanpur", 
                                                    "Nagpur", "Pune", "Indore", "Jaipur", "Dharamshala", 
                                                    "Thiruvananthapuram", "Guwahati", "Visakhapatnam", "Ranchi", 
                                                    "Lucknow", "Noida", "Cuttack", "Kochi", "Guwahati", "Trichy",
                                                   "Wankhede",'Raipur','Vadodara', 'Gwalior']
df['match_location'] = ''
for i in range(len(df.Location)):
    if df.Location.iloc[i] in indian_cities_with_international_cricket_grounds:
        df['match_location'].iloc[i] = 'Home'
    else:
        df['match_location'].iloc[i] = 'Away'


In [31]:
(df[df.match_location == 'Away']).Location.unique()

array(['Belfast', 'Brisbane', 'Melbourne', 'Canberra', 'Adelaide',
       'Sydney', 'Hobart', 'Mirpur', 'Karachi', 'Dambulla',
       'Colombo (RPS)', 'Hamilton', 'Auckland', 'Kingston', 'Gros Islet',
       'Bulawayo', 'Harare', 'Durban', 'Johannesburg', 'Cape Town',
       'Gqeberha', 'Centurion', 'Port of Spain', 'North Sound',
       'Chester-le-Street', 'Perth', 'Hambantota', 'Pallekele',
       'Dharamsala', 'Cardiff', 'The Oval', 'Birmingham', 'Napier',
       'Wellington', 'Fatullah', 'Eden Gardens', 'Nottingham', "Lord's",
       'Leeds', 'Dubai (DSC)', 'Brabourne', 'Mount Maunganui',
       'Southampton', 'Manchester', 'Providence', 'Bridgetown', 'Tarouba',
       'Christchurch', 'Paarl', 'Chattogram', 'Lahore', 'Abu Dhabi',
       'Sharjah', 'Dublin (Malahide)', 'Nelson', 'Dunedin', 'Bristol',
       'Taunton', 'Rawalpindi', 'Multan', 'Rotterdam'], dtype=object)

In [32]:
df.columns

Index(['player_id', 'player_name', 'runs_scored', 'wickets', 'runs_conceded',
       'catches', 'stumpings', 'opposition', 'match_id', 'opposition_team',
       'Location', 'match_location'],
      dtype='object')

In [33]:
df.corr()

,player_id,runs_scored,wickets,runs_conceded,catches,stumpings,match_id
player_id,1.000000,-0.158350,0.162327,0.178207,0.003962,0.029348,0.730379
runs_scored,-0.158350,1.000000,-0.321344,-0.400904,0.079816,0.064208,0.023148
wickets,0.162327,-0.321344,1.000000,0.558357,-0.091126,-0.040091,0.018036
runs_conceded,0.178207,-0.400904,0.558357,1.000000,-0.140269,-0.057623,-0.026254
catches,0.003962,0.079816,-0.091126,-0.140269,1.000000,0.051594,0.031140
stumpings,0.029348,0.064208,-0.040091,-0.057623,0.051594,1.000000,0.040592
match_id,0.730379,0.023148,0.018036,-0.026254,0.031140,0.040592,1.000000


In [34]:
df1 = pd.get_dummies(data=df,columns=['opposition_team','match_location'],drop_first=True)

In [35]:
df1.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,opposition,match_id,Location,...,opposition_team_Nepal,opposition_team_Netherlands,opposition_team_New Zealand,opposition_team_Pakistan,opposition_team_South Africa,opposition_team_Sri Lanka,opposition_team_U.A.E.,opposition_team_West Indies,opposition_team_Zimbabwe,match_location_Home
0,1,Rohit Sharma,0,0,0,1,0,v Ireland Belfast,1,Belfast,...,0,0,0,0,0,0,0,0,0,0
1,1,Rohit Sharma,8,0,3,1,0,v South Africa Belfast,2,Belfast,...,0,0,0,0,1,0,0,0,0,0
2,1,Rohit Sharma,1,0,0,0,0,v Australia Hyderabad,3,Hyderabad,...,0,0,0,0,0,0,0,0,0,1
3,1,Rohit Sharma,52,0,0,1,0,v Pakistan Jaipur,4,Jaipur,...,0,0,0,1,0,0,0,0,0,1
4,1,Rohit Sharma,29,0,0,0,0,v Australia Brisbane,5,Brisbane,...,0,0,0,0,0,0,0,0,0,0


In [36]:
player_dict = df[['player_id','player_name']].drop_duplicates().set_index('player_id').to_dict()
print(player_dict)

{'player_name': {1: 'Rohit Sharma', 4: 'Virat Kohli', 7: 'Ravindra Jadeja', 13: 'Ravichandran ashwin', 12: 'Mohammed Shami', 9: 'Jasprit Bumrah', 2: 'Hardik Pandya', 16: 'Babar Azam', 17: 'Shadab Khan', 19: 'Faqar Zaman', 6: 'KL Rahul', 8: 'Shardul Thakur', 11: 'Kuldeep Yadav', 5: 'Shreyas Iyer', 23: 'Imam-Ul-Haq', 24: 'Mohammad Nawaz', 29: 'Shaheen Shah Afridi', 10: 'Mohammed Siraj', 3: 'Shubman Gill', 14: 'Ishan Kishan', 15: 'Suryakumar Yadav', 20: 'Haris Rauf', 22: 'Iftikhar Ahmed', 25: 'Mohammad Rizwan', 27: 'Salman Agha', 28: 'Saud Shakeel', 21: 'Hasan Ali', 26: 'Mohammad Wasim Jr.', 18: 'Abdullah Shafique', 30: 'Usama Mir'}}


In [37]:
df1.drop(columns={'player_name','opposition','runs_conceded','catches','stumpings','Location'},inplace=True)

In [38]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   player_id                     2021 non-null   int64
 1   runs_scored                   2021 non-null   int64
 2   wickets                       2021 non-null   int64
 3   match_id                      2021 non-null   int64
 4   opposition_team_Australia     2021 non-null   uint8
 5   opposition_team_Bangladesh    2021 non-null   uint8
 6   opposition_team_England       2021 non-null   uint8
 7   opposition_team_Hong Kong     2021 non-null   uint8
 8   opposition_team_India         2021 non-null   uint8
 9   opposition_team_Ireland       2021 non-null   uint8
 10  opposition_team_Nepal         2021 non-null   uint8
 11  opposition_team_Netherlands   2021 non-null   uint8
 12  opposition_team_New Zealand   2021 non-null   uint8
 13  opposition_team_Pakistan      202

In [91]:
X_runs = df1.drop(columns = {'runs_scored','wickets'})
y_runs = np.log1p(df1[['runs_scored']])

In [92]:
X_train,X_test,y_train,y_test = train_test_split(X_runs,y_runs,test_size=0.2,random_state=42)

In [93]:
runs_train_data_combined = X_train.copy()
runs_train_data_combined['runs_scored'] = y_train

In [94]:
fit_data = setup(data = runs_train_data_combined,target = 'runs_scored')
best = compare_models()

,Description,Value
0,Session id,6482
1,Target,runs_scored
2,Target type,Regression
3,Original data shape,"(1616, 19)"
4,Transformed data shape,"(1616, 19)"
5,Transformed train set shape,"(1131, 19)"
6,Transformed test set shape,"(485, 19)"
7,Numeric features,18
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,1.1426,1.9122,1.3817,0.3252,0.5560,0.4325,0.0200
lightgbm,Light Gradient Boosting Machine,1.1374,1.9660,1.4006,0.3055,0.5516,0.4640,0.0630
ada,AdaBoost Regressor,1.2178,2.0167,1.4189,0.2897,0.5834,0.4102,0.0120
rf,Random Forest Regressor,1.1734,2.2183,1.4872,0.2155,0.5804,0.4854,0.0550
knn,K Neighbors Regressor,1.2840,2.4641,1.5686,0.1301,0.6188,0.4633,0.0130
br,Bayesian Ridge,1.4304,2.6555,1.6290,0.0638,0.6718,0.4507,0.0080
en,Elastic Net,1.4391,2.6684,1.6330,0.0591,0.6745,0.4478,0.0080
ridge,Ridge Regression,1.4263,2.6725,1.6341,0.0581,0.6712,0.4573,0.0100
lr,Linear Regression,1.4263,2.6786,1.6360,0.0559,0.6717,0.4590,0.0090
lasso,Lasso Regression,1.4496,2.6962,1.6415,0.0494,0.6780,0.4475,0.0110


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

Let us create an esemble model of the top 5 regressors by assigning weights to each one of them, in the order of reducing ranks.

In [95]:
models_runs = {
    'ada' : AdaBoostRegressor(),
    'Gradient_boosting_reg' : GradientBoostingRegressor(),
    'Bayesian_Ridge' : BayesianRidge(),
    'Light_Gradient_Boosting' : LGBMRegressor(),
    'Random_forest_regression' : RandomForestRegressor() 
}

In [96]:
for name, model in models_runs.items():
    model.fit(X_train,y_train)
    print(name,"is trained")

ada is trained
Gradient_boosting_reg is trained
Bayesian_Ridge is trained
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000062 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 309
[LightGBM] [Info] Number of data points in the train set: 1616, number of used features: 14
[LightGBM] [Info] Start training from score 2.165225
Light_Gradient_Boosting is trained
Random_forest_regression is trained


In [97]:
for model_name,model in models_runs.items():
    model_result = np.exp(np.sqrt(-cross_val_score(estimator = model,X=X_train,y=y_train,cv=5, 
                                   scoring = 'neg_root_mean_squared_error')))
    print(model_name,'\n')
    print('Mean: ',model_result.mean(),'\n')
    print('Std: ',model_result.std(),'\n')

ada 

Mean:  3.3292943395025913 

Std:  0.05849717890075975 

Gradient_boosting_reg 

Mean:  3.2667967080009435 

Std:  0.09639281396954681 

Bayesian_Ridge 

Mean:  3.595487390850605 

Std:  0.025956301606755522 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000058 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 1292, number of used features: 14
[LightGBM] [Info] Start training from score 2.156289
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000052 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 300
[LightGBM] [Info] Number of data points in the train set: 1293, number of used features: 1

In [99]:
y_pred_runs = np.around(np.exp(
        0.3 * models_runs['ada'].predict(X_test) + 
        0.3 * models_runs['Gradient_boosting_reg'].predict(X_test) + 
        0.2 * models_runs['Light_Gradient_Boosting'].predict(X_test) + 
        0.1 * models_runs['Random_forest_regression'].predict(X_test) + 
        0.1 * models_runs['Bayesian_Ridge'].predict(X_test))+1,decimals=0)

In [100]:
y_pred_runs = np.log1p(y_pred_runs)

In [101]:
mean_squared_error(y_test,y_pred_runs)

2.1969095705205297

<h3> Now building model to predict wickets secured by each player

In [102]:
X_wickets = df1.drop(columns = {'wickets','runs_scored'})
y_wickets = df1[['wickets']]

In [103]:
X_train,X_test,y_train,y_test = train_test_split(X_wickets,y_wickets,test_size=0.2,random_state=42)

In [104]:
wickets_train_data_combined = X_train.copy()
wickets_train_data_combined['wickets'] = y_train
fit_data = setup(data = wickets_train_data_combined,target = 'wickets')
best = compare_models()

,Description,Value
0,Session id,7397
1,Target,wickets
2,Target type,Regression
3,Original data shape,"(1616, 19)"
4,Transformed data shape,"(1616, 19)"
5,Transformed train set shape,"(1131, 19)"
6,Transformed test set shape,"(485, 19)"
7,Numeric features,18
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.5634,0.8495,0.9167,0.3362,0.4048,0.4561,0.0200
lightgbm,Light Gradient Boosting Machine,0.5965,0.9159,0.9540,0.2743,0.4257,0.5155,0.0510
rf,Random Forest Regressor,0.5789,1.0291,1.0115,0.1868,0.4393,0.5890,0.0520
ada,AdaBoost Regressor,0.7518,1.0975,1.0452,0.1287,0.5324,0.4744,0.0120
et,Extra Trees Regressor,0.6130,1.2140,1.0966,0.0427,0.4735,0.6835,0.0390
knn,K Neighbors Regressor,0.7318,1.2153,1.0996,0.0349,0.5216,0.6090,0.0110
ridge,Ridge Regression,0.8340,1.2342,1.1065,0.0333,0.5376,0.5427,0.0080
br,Bayesian Ridge,0.8455,1.2338,1.1066,0.0326,0.5375,0.5404,0.0080
lr,Linear Regression,0.8355,1.2467,1.1116,0.0247,0.5395,0.5445,0.0090
lar,Least Angle Regression,0.8358,1.2483,1.1123,0.0236,0.5401,0.5461,0.0080


Processing:   0%|          | 0/77 [00:00<?, ?it/s]

In [105]:
models_wickets = {
    'ada' : AdaBoostRegressor(),
    'Gradient_boosting_reg' : GradientBoostingRegressor(),
    'Bayesian_Reidge' : BayesianRidge(),
    'Ridge_Regression' : Ridge(),
    'Light_Gradient_Boosting' : LGBMRegressor(),
    'Random_forest_regression' : RandomForestRegressor() 
}

In [106]:
for name, model in models_wickets.items():
    model.fit(X_train,y_train)
    print(name,"is trained")

ada is trained
Gradient_boosting_reg is trained
Bayesian_Reidge is trained
Ridge_Regression is trained
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 309
[LightGBM] [Info] Number of data points in the train set: 1616, number of used features: 14
[LightGBM] [Info] Start training from score 0.666460
Light_Gradient_Boosting is trained
Random_forest_regression is trained


In [107]:
for model_name,model in models_wickets.items():
    model_result = np.exp(np.sqrt(-cross_val_score(estimator = model,X=X_train,y=y_train,cv=5, 
                                   scoring = 'neg_root_mean_squared_error')))
    print(model_name,'\n')
    print('Mean: ',model_result.mean(),'\n')
    print('Std: ',model_result.std(),'\n')

ada 

Mean:  2.7482601792284678 

Std:  0.09749444227415885 

Gradient_boosting_reg 

Mean:  2.5884882247639003 

Std:  0.05520337357142509 

Bayesian_Reidge 

Mean:  2.847591906556874 

Std:  0.046467129794814516 

Ridge_Regression 

Mean:  2.848209628987373 

Std:  0.0465423245521746 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000062 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 1292, number of used features: 14
[LightGBM] [Info] Start training from score 0.666409
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000050 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 300
[LightGBM] [Info

In [108]:
y_pred_wickets = np.around(
        0.2 * models_wickets['ada'].predict(X_test) + 
        0.3 * models_wickets['Gradient_boosting_reg'].predict(X_test) + 
        0.3 * models_wickets['Light_Gradient_Boosting'].predict(X_test) + 
        0.1 * models_wickets['Random_forest_regression'].predict(X_test) + 
        0.1 * models_wickets['Bayesian_Reidge'].predict(X_test),decimals=0)

In [109]:
y_pred_wickets[:10]

array([ 2.,  2.,  1.,  0.,  1.,  0.,  1.,  0.,  0., -0.])

In [117]:
df1.columns

Index(['player_id', 'runs_scored', 'wickets', 'match_id',
       'opposition_team_Australia', 'opposition_team_Bangladesh',
       'opposition_team_England', 'opposition_team_Hong Kong',
       'opposition_team_India', 'opposition_team_Ireland',
       'opposition_team_Nepal', 'opposition_team_Netherlands',
       'opposition_team_New Zealand', 'opposition_team_Pakistan',
       'opposition_team_South Africa', 'opposition_team_Sri Lanka',
       'opposition_team_U.A.E.', 'opposition_team_West Indies',
       'opposition_team_Zimbabwe', 'match_location_Home'],
      dtype='object')

In [118]:
X_columns = list(df1.columns)
X_columns.remove('runs_scored')
X_columns.remove('wickets')

In [135]:
result_runs = pd.DataFrame(columns=X_columns)

In [136]:
result_runs.player_id = [i for i in range(1,31)]
result_runs.head()

,player_id,match_id,opposition_team_Australia,opposition_team_Bangladesh,opposition_team_England,opposition_team_Hong Kong,opposition_team_India,opposition_team_Ireland,opposition_team_Nepal,opposition_team_Netherlands,opposition_team_New Zealand,opposition_team_Pakistan,opposition_team_South Africa,opposition_team_Sri Lanka,opposition_team_U.A.E.,opposition_team_West Indies,opposition_team_Zimbabwe,match_location_Home
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
result_runs.columns

Index(['player_id', 'match_id', 'opposition_team_Australia',
       'opposition_team_Bangladesh', 'opposition_team_England',
       'opposition_team_Hong Kong', 'opposition_team_India',
       'opposition_team_Ireland', 'opposition_team_Nepal',
       'opposition_team_Netherlands', 'opposition_team_New Zealand',
       'opposition_team_Pakistan', 'opposition_team_South Africa',
       'opposition_team_Sri Lanka', 'opposition_team_U.A.E.',
       'opposition_team_West Indies', 'opposition_team_Zimbabwe',
       'match_location_Home'],
      dtype='object')

In [143]:
result_runs.match_id = df1.match_id.max() + 1
columns_0 = ['opposition_team_Australia',
       'opposition_team_Bangladesh', 'opposition_team_England',
       'opposition_team_Hong Kong', 'opposition_team_India',
       'opposition_team_Ireland', 'opposition_team_Nepal',
       'opposition_team_Netherlands', 'opposition_team_New Zealand', 'opposition_team_South Africa',
       'opposition_team_Sri Lanka', 'opposition_team_U.A.E.',
       'opposition_team_West Indies', 'opposition_team_Zimbabwe']
for i in columns_0:
    result_runs[i] = 0
result_runs.opposition_team_Pakistan = 1
result_runs.match_location_Home = 1
result_runs.head()

,player_id,match_id,opposition_team_Australia,opposition_team_Bangladesh,opposition_team_England,opposition_team_Hong Kong,opposition_team_India,opposition_team_Ireland,opposition_team_Nepal,opposition_team_Netherlands,opposition_team_New Zealand,opposition_team_Pakistan,opposition_team_South Africa,opposition_team_Sri Lanka,opposition_team_U.A.E.,opposition_team_West Indies,opposition_team_Zimbabwe,match_location_Home
0,1,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,2,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,3,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,4,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,5,475,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [149]:
X_runs_wickets = df1[X_columns]
y_runs = np.log1p(df1['runs_scored'])
y_wickets = df1['wickets']

In [150]:
for name, model in models_runs.items():
    model.fit(X_runs_wickets,y_runs)
    print(name,"is trained")

ada is trained
Gradient_boosting_reg is trained
Bayesian_Ridge is trained
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000157 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 312
[LightGBM] [Info] Number of data points in the train set: 2021, number of used features: 15
[LightGBM] [Info] Start training from score 2.157867
Light_Gradient_Boosting is trained
Random_forest_regression is trained


In [151]:
for model_name,model in models_runs.items():
    model_result = np.exp(np.sqrt(-cross_val_score(estimator = model,X=X_runs_wickets,y=y_runs,cv=5, 
                                   scoring = 'neg_root_mean_squared_error')))
    print(model_name,'\n')
    print('Mean: ',model_result.mean(),'\n')
    print('Std: ',model_result.std(),'\n')

ada 

Mean:  3.3559319393135025 

Std:  0.11654170769965312 

Gradient_boosting_reg 

Mean:  3.3824450613471795 

Std:  0.14481056158622457 

Bayesian_Ridge 

Mean:  3.7376561908987016 

Std:  0.12281016238488628 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000073 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 308
[LightGBM] [Info] Number of data points in the train set: 1616, number of used features: 15
[LightGBM] [Info] Start training from score 2.153798
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 310
[LightGBM] [Info] Number of data points in the train set: 1617, number of used features: 1

In [152]:
y_pred_runs = np.around(np.exp(
        0.3 * models_runs['ada'].predict(result_runs) + 
        0.3 * models_runs['Gradient_boosting_reg'].predict(result_runs) + 
        0.2 * models_runs['Light_Gradient_Boosting'].predict(result_runs) + 
        0.1 * models_runs['Random_forest_regression'].predict(result_runs) + 
        0.1 * models_runs['Bayesian_Ridge'].predict(result_runs))+1,decimals=0)

In [153]:
for name, model in models_wickets.items():
    model.fit(X_runs_wickets,y_wickets)
    print(name,"is trained")

ada is trained
Gradient_boosting_reg is trained
Bayesian_Reidge is trained
Ridge_Regression is trained
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 312
[LightGBM] [Info] Number of data points in the train set: 2021, number of used features: 15
[LightGBM] [Info] Start training from score 0.671945
Light_Gradient_Boosting is trained
Random_forest_regression is trained


In [154]:
for model_name,model in models_wickets.items():
    model_result = np.exp(np.sqrt(-cross_val_score(estimator = model,X=X_train,y=y_train,cv=5, 
                                   scoring = 'neg_root_mean_squared_error')))
    print(model_name,'\n')
    print('Mean: ',model_result.mean(),'\n')
    print('Std: ',model_result.std(),'\n')

ada 

Mean:  2.7611754376904414 

Std:  0.10307228516749996 

Gradient_boosting_reg 

Mean:  2.5887066356276085 

Std:  0.05482121688154046 

Bayesian_Reidge 

Mean:  2.847591906556874 

Std:  0.046467129794814516 

Ridge_Regression 

Mean:  2.848209628987373 

Std:  0.0465423245521746 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000077 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 306
[LightGBM] [Info] Number of data points in the train set: 1292, number of used features: 14
[LightGBM] [Info] Start training from score 0.666409
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000062 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 300
[LightGBM] [Info

In [155]:
y_pred_wickets = np.around(
        0.2 * models_wickets['ada'].predict(result_runs) + 
        0.3 * models_wickets['Gradient_boosting_reg'].predict(result_runs) + 
        0.3 * models_wickets['Light_Gradient_Boosting'].predict(result_runs) + 
        0.1 * models_wickets['Random_forest_regression'].predict(result_runs) + 
        0.1 * models_wickets['Bayesian_Reidge'].predict(result_runs),decimals=0)

In [156]:
y_pred_runs

array([17., 15., 22., 19., 18., 18.,  9.,  8.,  4.,  4.,  4.,  4.,  5.,
       21., 21., 21.,  8., 12., 13.,  4.,  5., 12., 15.,  9., 12.,  5.,
       10.,  8.,  4.,  4.])

In [157]:
y_pred_wickets

array([0., 1., 0., 0., 0., 1., 2., 2., 2., 2., 2., 2., 2., 1., 1., 1., 1.,
       1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.])

In [160]:
pred = []
for i, j  in zip(range(len(y_pred_runs)),range(len(y_pred_wickets))):
    pred.append([y_pred_runs[i],y_pred_wickets[j]])
    
pred

[[17.0, 0.0],
 [15.0, 1.0],
 [22.0, 0.0],
 [19.0, 0.0],
 [18.0, 0.0],
 [18.0, 1.0],
 [9.0, 2.0],
 [8.0, 2.0],
 [4.0, 2.0],
 [4.0, 2.0],
 [4.0, 2.0],
 [4.0, 2.0],
 [5.0, 2.0],
 [21.0, 1.0],
 [21.0, 1.0],
 [21.0, 1.0],
 [8.0, 1.0],
 [12.0, 1.0],
 [13.0, 1.0],
 [4.0, 2.0],
 [5.0, 1.0],
 [12.0, 1.0],
 [15.0, 1.0],
 [9.0, 1.0],
 [12.0, 1.0],
 [5.0, 1.0],
 [10.0, 1.0],
 [8.0, 1.0],
 [4.0, 2.0],
 [4.0, 2.0]]

In [161]:
for i in range(len(pred)):
    for j in range(0,2):
        pred[i][j] = int(round(pred[i][j]))

In [162]:
final_results = pd.DataFrame(pred)
final_results.head()

,0,1
0,17,0
1,15,1
2,22,0
3,19,0
4,18,0


In [163]:
final_results['player_id'] = [i for i in range(1,31)]
final_results.head()

,0,1,player_id
0,17,0,1
1,15,1,2
2,22,0,3
3,19,0,4
4,18,0,5


In [164]:
final_results.rename(columns={0:'runs',1:'wickets'},inplace = True)

In [165]:
final_results.head()

,runs,wickets,player_id
0,17,0,1
1,15,1,2
2,22,0,3
3,19,0,4
4,18,0,5


In [166]:
final_results = final_results.iloc[:,[2,0,1]]
final_results.head()

,player_id,runs,wickets
0,1,17,0
1,2,15,1
2,3,22,0
3,4,19,0
4,5,18,0


In [167]:
final_results.to_csv('submission1.csv',index = False)

In [ ]:

+